In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.cm import get_cmap
from matplotlib.colors import BoundaryNorm
import numpy as np

# Point Source vs. Line Source

# Spherical Wave vs. Cylindrical Wave

In [ ]:
def plot_soundfield(p):

    col_tick = np.linspace(-2, +2, 255, endpoint=True)
    cmap_lin = get_cmap('bwr').copy()
    norm_lin = BoundaryNorm(col_tick, cmap_lin.N)

    dBmax = 12
    dBmin = -30
    dBstep = 3

    col_tick = np.linspace(dBmin, dBmax, (dBmax - dBmin) //
                           dBstep + 1, endpoint=True)
    cmap_dB = get_cmap('viridis').copy()
    norm_dB = BoundaryNorm(col_tick, cmap_dB.N)

    fig, ax = plt.subplots(figsize=(12, 6),
                           nrows=1,
                           ncols=2,
                           subplot_kw=dict(box_aspect=1))  # equal axis scaling

    cax_lin = fig.add_axes([0.15, 0.2, 0.02, 0.6])

    cax_dB = fig.add_axes([0.55, 0.2, 0.02, 0.6])

    surf_lin = ax[0].pcolormesh(x, y, np.real(p),
                                cmap=cmap_lin,
                                norm=norm_lin,
                                antialiased=True)

    surf_dB = ax[1].pcolormesh(x, y, 10*np.log10(np.abs(p)**2 / 1**2),
                               cmap=cmap_dB,
                               norm=norm_dB,
                               antialiased=True)

    fig.colorbar(surf_lin,
                 cax=cax_lin,
                 ticks=np.linspace(-2, 2, 5),
                 orientation='vertical',
                 label='lin')

    fig.colorbar(surf_dB,
                 cax=cax_dB,
                 ticks=np.linspace(
                     dBmax, dBmin, (dBmax - dBmin) // (2*dBstep) + 1),
                 orientation='vertical',
                 label='dB(rel1)')

    for i in range(2):
        ax[i].set_xticks(np.linspace(-8, 8, 9))
        ax[i].set_yticks(np.linspace(-8, 8, 9))
        ax[i].set_xlabel('x / m')
        ax[i].set_ylabel('y / m')
        ax[i].grid(True)

## Acoustic Parameters

In [ ]:
c = 343  # m/s, speed of sound
wave_length = 2  # m
w_c = 2*np.pi / wave_length  # rad/m
w = w_c*c  # rad/s, angular frequency

## Square xy-Plane as Grid

In [ ]:
N = 2**8
xylim = 8
x, y = np.meshgrid(np.linspace(-xylim, xylim, N, endpoint=True),
                   np.linspace(-xylim, xylim, N, endpoint=True),
                   indexing='ij')
xvg = np.array([x, y])  # prepare for broadcasting

## Point Source Position within z=0 Plane

In [ ]:
xps = 0  # m
yps = 2  # m
xpsv = np.array([xps, yps])
xpsv = xpsv[:, np.newaxis, np.newaxis]  # prepare dim for broadcasting
xpsv.shape

In [ ]:
r = np.linalg.norm(xvg - xpsv, axis=0)  # works due to broadcasting
# outgoing spherical wave with exp(+j w t) time convention
p = np.exp(-1j * w_c * r) / (4*np.pi*r)
p_norm = 4*np.pi
p *= p_norm
plot_soundfield(p)

## Line Source, invariant w.r.t z

In [ ]:
xls = 0  # m
yls = 2  # m
xlsv = np.array([xls, yls])
xlsv = xlsv[:, np.newaxis, np.newaxis]  # prepare dim for broadcasting
xlsv.shape

In [ ]:
r = np.linalg.norm(xvg - xlsv, axis=0)  # works due to broadcasting

### Bessel Differential Equation

is the wave equation solution for the uniformly driven line source with respect to the radial component.
See https://dlmf.nist.gov/10.2#i

We need either terms including $ \propto J_0(\frac{\omega}{c}r) - \mathrm{j} Y_0(\frac{\omega}{c}r)$, which is known as Bessel function of the first and second kind, respectively, of order 0.
Alternatively, we can deploy $H^{(2)}_0(\frac{\omega}{c}r) = J_0(\frac{\omega}{c}r) - \mathrm{j} Y_0(\frac{\omega}{c}r)$ directly as a function. $H^{(2)}_0(\cdot)$ is known as Hankel function of 2nd kind of order 0 or sometimes Bessel function of the third kind.

Note that Hankel function of 2nd kind is strictly linked to our used $\mathrm{e}^{+ \mathrm{j} \omega t}$ time convention.

We make sure, that we don't confuse this with another option to describe wave propagation $H^{(1)}_0(\cdot) \mathrm{e}^{- \mathrm{j} \omega t}$, using Hankel function of first kind of order zero $H^{(1)}_0(\frac{\omega}{c}r) = J_0(\frac{\omega}{c}r) + \mathrm{j} Y_0(\frac{\omega}{c}r)$



In [ ]:
from scipy.special import jv, yv, hankel2
# check equality with an example
np.allclose(jv(0, np.pi) - 1j*yv(0, np.pi), hankel2(0, np.pi))

In [ ]:
# outgoing cylindrical wave with exp(+j w t) time convention
p = -1j / 4 * hankel2(0, w_c * r)
p_norm = np.sqrt(8*np.pi*w_c) * np.exp(-1j*7*np.pi/4)
p *= p_norm
plot_soundfield(p)

### Pressure Decay of Line Source along y-Axis

In [ ]:
y1 = np.linspace(-8, 8, 1000)
xv = np.array([0*y1, y1])
print(xv.shape)

xlsv = np.array([xls, yls])
xlsv = xlsv[:, np.newaxis]  # prepare dim for broadcasting
print(xlsv.shape)

r = np.linalg.norm(xv - xlsv, axis=0)  # works due to broadcasting
# outgoing cylindrical wave with exp(+j w t) time convention
p = -1j / 4 * hankel2(0, w_c * r)
r_norm = 2  # m
p_norm = np.sqrt(8*np.pi*w_c * r_norm) * np.exp(-1j*7*np.pi/4)
p *= p_norm

# large argument approximation of hankel2(0, ...) yields
p_farfield_highfrequency = -1j / 4 * \
    np.sqrt(2/(np.pi*w_c*r)) * np.exp(-1j*w_c*r) * np.exp(+1j*np.pi/4)
p_farfield_highfrequency *= p_norm  # same p_norm as above

plt.figure(figsize=(8, 5))
plt.plot(y1, np.real(p), 'C0', label='exact')
plt.plot(y1, np.real(p_farfield_highfrequency),
         'C1:', label='large arg approx')
plt.plot(4, 1, 'C3o')
plt.plot(0, 1, 'C3o')
plt.xticks(np.arange(-8, 9))
plt.yticks(np.arange(-2, 9))
plt.ylim(-2, 9)
plt.xlabel('y / m')
plt.ylabel('pressure')
plt.legend(loc='upper left')
plt.grid(True)

<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <span rel="dct:publisher" resource="[_:publisher]">the person who associated CC0</span>
  with this work has waived all copyright and related or neighboring
    rights to this work.
</p>